## DCF_1 ###

- Basic version of DCF 
- Acquire data from Yahoo finance using "yfinance"


step :
- get > Average Free cash flow , shares , Net Debt = Long Term Borrowings - Cash
- input > growth rate , terminal growth rate , discount rate , horizons
- calculate > FCF 
- calculate > terminal value
- calculate > present value
- calculate > Intrinsic value
- calculate > Margin of safety
- summary

In [56]:
import pandas as pd
import yfinance as yf

In [57]:
ticker = 'hmpro.bk'
data = yf.Ticker(ticker)
data.info

{'address1': '31 Prachachuen-Nonthaburi Road',
 'address2': 'Bangkhen Amphoe Muang',
 'city': 'Nonthaburi',
 'zip': '11000',
 'country': 'Thailand',
 'phone': '66 2 832 1000',
 'fax': '66 2 832 1234',
 'website': 'https://www.homepro.co.th',
 'industry': 'Home Improvement Retail',
 'industryKey': 'home-improvement-retail',
 'industryDisp': 'Home Improvement Retail',
 'sector': 'Consumer Cyclical',
 'sectorKey': 'consumer-cyclical',
 'sectorDisp': 'Consumer Cyclical',
 'longBusinessSummary': 'Home Product Center Public Company Limited operates as a home improvement retailer in Thailand, Malaysia, and Vietnam. The company trades in various goods and materials for construction, addition, refurbishment, renovation, and improvement of buildings, houses, and residences; and offers related services to retail businesses. It also provides installation, maintenance, warehousing management and distribution, and other services; and utilities services. In addition, the company rents space; and enga

In [58]:
### - get > Average Free cash flow , shares , Net Debt = Long Term Borrowings - Cash ###

## Shares ## 
shares = data.info['sharesOutstanding']
print('Ticker = ',ticker,' Shares = ',shares)


## Net Debt = Long Term Borrowings - Cash ###
totalnoncurrentliabilities = data.balancesheet.loc[data.balancesheet.index == 'Total Non Current Liabilities Net Minority Interest'].values[0][0] - data.balancesheet.loc[data.balancesheet.index == 'Cash And Cash Equivalents'].values[0][0]
print('Ticker = ',ticker,' Net Debt = ',totalnoncurrentliabilities)

## Average Free Cash Flow ## 
averagefreecashflow = data.cash_flow.T['Free Cash Flow'].dropna().mean()
print(data.cash_flow.T['Free Cash Flow'].dropna())
print('Ticker = ',ticker,'Average Free Cash Flow = ',averagefreecashflow)

Ticker =  hmpro.bk  Shares =  13151200256
Ticker =  hmpro.bk  Net Debt =  13481168768.0
2023-12-31    4561597342.0
2022-12-31    4364554238.0
2021-12-31    6265215288.0
2020-12-31    6055262093.0
Name: Free Cash Flow, dtype: object
Ticker =  hmpro.bk Average Free Cash Flow =  5311657240.25


In [59]:
## - input > growth rate , terminal growth rate , discount rate , horizons ##
FCF = data.cash_flow.T['Free Cash Flow'].dropna()
FCF = pd.DataFrame(FCF)
FCF = FCF.sort_index(ascending=True)
FCF['diff'] = FCF['Free Cash Flow'].diff(1)
FCF['growth'] = FCF['diff']/FCF['Free Cash Flow']
print(FCF)
growthrate = FCF['growth'].mean()
print("Average Growth Rate from Operating Cash Flow = ",growthrate)

year_growth = 3
year_sustain = 7
discountrate = 0.10
terminalgrowthrate = 0.02
growthrate = 0.20

           Free Cash Flow          diff    growth
2020-12-31   6055262093.0           NaN       NaN
2021-12-31   6265215288.0   209953195.0  0.033511
2022-12-31   4364554238.0 -1900661050.0 -0.435477
2023-12-31   4561597342.0   197043104.0  0.043196
Average Growth Rate from Operating Cash Flow =  -0.11958985004147378


In [60]:
 ## - calculate > FCF ##

FCF_year_growth = [averagefreecashflow * ((1+growthrate)**i) for i in range(1,year_growth+1,1)]
print("FCF year growth",FCF_year_growth)

FCF_year_sustain = [FCF_year_growth[-1] * ((1+(growthrate/2))**i) for i in range(1,year_sustain+1,1)]
print("FCF year sustain",FCF_year_sustain)

## calculate > terminal value
terminalvalue = FCF_year_sustain[-1] * (1+(terminalgrowthrate)) / (discountrate-terminalgrowthrate)
print(FCF_year_sustain[-1]* (1+(terminalgrowthrate)))
print("Terminal value = ",terminalvalue)


FCF year growth [np.float64(6373988688.3), np.float64(7648786425.96), np.float64(9178543711.151999)]
FCF year sustain [np.float64(10096398082.267199), np.float64(11106037890.49392), np.float64(12216641679.543314), np.float64(13438305847.497644), np.float64(14782136432.24741), np.float64(16260350075.472153), np.float64(17886385083.01937)]
18244112784.67976
Terminal value =  228051409808.497


In [61]:
## - calculate > present value ##
FCF_year_growth = FCF_year_growth + (FCF_year_sustain)
FCF_year_growth.append(terminalvalue)
pv = pd.DataFrame(FCF_year_growth,columns=['FCF'])
pv['pv'] = [pv['FCF'][i]/((1+discountrate)**(i+1)) for i in range(len(pv))]
pv['pv'][pv.index[-1]] = pv['FCF'][pv.index[-1]] / ((1+discountrate)**(len(pv)-1))
print(pv)

totalpv = pv['pv'].sum() - totalnoncurrentliabilities
print('total pv = ',totalpv)

intrinsicvalue = totalpv/shares
print('intrinsicvalue = ',intrinsicvalue)

             FCF            pv
0   6.373989e+09  5.794535e+09
1   7.648786e+09  6.321311e+09
2   9.178544e+09  6.895976e+09
3   1.009640e+10  6.895976e+09
4   1.110604e+10  6.895976e+09
5   1.221664e+10  6.895976e+09
6   1.343831e+10  6.895976e+09
7   1.478214e+10  6.895976e+09
8   1.626035e+10  6.895976e+09
9   1.788639e+10  6.895976e+09
10  2.280514e+11  8.792369e+10
total pv =  141726174122.91132
intrinsicvalue =  10.77667219448288


C:\Users\595078\AppData\Local\Temp\ipykernel_4548\3552423011.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  pv['pv'][pv.index[-1]] = pv['FCF'][pv.index[-1]] / ((1+discountrate)**(len(pv)-1))
